In [1]:
## David_Labuschagne_Classification_ Predict

In [2]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
test = pd.read_csv("../input/climate-change/test.csv")
train = pd.read_csv("../input/climate-change/train.csv")

In [4]:
train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [5]:
train.sentiment.value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

Normally we would need to balance the DATA because of the high number of Pro (1) tweets. However, this negaatively effected the score and has therefore been ommitted. 


In [6]:
#Check for missing values
print(train.isnull().sum())
print(train.isnull().sum())

sentiment    0
message      0
tweetid      0
dtype: int64
sentiment    0
message      0
tweetid      0
dtype: int64


**Data Cleaning**
When removing stop words, hastags etc. as well as doing lemmatization I found the model performed worse then leaving the raw tweets.

In [7]:
# Remove punctuation
#import string
#print ('Cleaning punctuation...')
#def remove_punctuation_numbers(post):
    #punc_numbers = string.punctuation + '0123456789'
    #return ''.join([l for l in post if l not in punc_numbers])#train['message'] = train['message'].apply(remove_punctuation_numbers)
#test['message'] = test['message'].apply(remove_punctuation_numbers)

Punctuation removal negatively effected the score. 


In [8]:
## Remove urls
print ('Removing URLs...')
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
train['message'] = train['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)
test['message'] = test['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

Removing URLs...


In [9]:
# Make lower case
#print ('Lowering case...')
#train['message'] = train['message'].str.lower()
#test['message'] = test['message'].str.lower()

In [10]:
train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [11]:
y = train['sentiment']
X = train['message']

In [12]:
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

Train/Test Split

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_vectorized,
                                                y,
                                                test_size=0.2, 
                                                 random_state=3)

Linear Support Vector Classification Model

In [14]:
svc = LinearSVC( C=1, random_state=0)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_val)

In [15]:
f1_score(y_val, y_pred, average="macro")

0.6700446990370189

In [16]:
#Hyperparameter Tuning for SVC
parameters = {'C':(0.25,1.0, 1.2),
              'random_state': (0, 3, 42, 101),
             }

In [17]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svc, parameters)
clf.fit(X_train,y_train)

GridSearchCV(estimator=LinearSVC(C=1, random_state=0),
             param_grid={'C': (0.25, 1.0, 1.2),
                         'random_state': (0, 3, 42, 101)})

In [18]:
clf.best_params_

{'C': 1.0, 'random_state': 0}

In [19]:
#K-fold cross validation 
from sklearn.model_selection import cross_val_score
print(cross_val_score(svc, X_train, y_train, scoring='f1_macro'))

[0.65875316 0.64880325 0.63376762 0.64963192 0.6575259 ]


Preparing the test set. 

In [20]:
testx = test['message']
test_vect = vectorizer.transform(testx)


In [21]:
y_pred = svc.predict(test_vect)

In [22]:
test['sentiment'] = y_pred

In [23]:
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,1


Kaggle Submission 

In [24]:
test[['tweetid','sentiment']].to_csv('submissionclass14.csv', index=False)